In [ ]:
import pandas as pd
import numpy as np
import os

print(os.getcwd())
# controleer altijd of je in de map helpfiles zit

In [ ]:
from select_df import select_df

df_foodprices = select_df(1, 'foodprices')
df_migration = select_df(1, 'migration_movements')
df_weather = select_df(1, 'temperature_and_precipitation')

# Plot migration movements

In [ ]:
def select_plot_migration_movements(input_data,origin, year1, year2):
    input_data = input_data.loc[input_data['Origin'] == origin]
    input_data = input_data.loc[(input_data['Timestamp'] > year1) & (input_data['Timestamp'] < year2)]
    return input_data

In [ ]:
test = select_plot_migration_movements(df_migration, 'Zimbabwe', 1999, 2017)
test

# Plot temperature and precipitation

In [ ]:
# geeft voor een #input_data van een #country de pr en tas vanaf #year1 tot en met #year2

def select_plot_temperature_and_precipitation(input_data, country, year1, year2):
    saved_dict = {'Mauritania': 'MRT', 'Lesotho': 'LSO', 'Somalia': 'SOM', 'Nigeria': 'NGA', 'Tanzania': 'TZA', 'Zambia': 'ZMB', 'Burundi': 'BDI', 'Afghanistan': 'AFG', 'Mali': 'MLI', 'Niger': 'NER', 'Malawi': 'MWI', 'Congo': 'ZAR', 'Cabo Verde': 'CPV', 'Sudan': 'SDN', 'Pakistan': 'PAK', 'Burkina Faso': 'BFA', 'Rwanda': 'RWA', 'Kenya': 'KEN', 'Senegal': 'SEN', 'Cameroon': 'CMR', 'Sierra Leone': 'SLE', 'Iraq': 'IRQ', 'Uganda': 'UGA', 'Mozambique': 'MOZ', 'Zimbabwe': 'ZWE', 'Central African Republic': 'CAF', 'Ethiopia': 'ETH', 'Guinea': 'GIN', 'Liberia': 'LBR', 'Djibouti': 'DJI', 'Iran': 'IRN', 'Madagascar': 'MDG', 'Lebanon': 'LBN'}

    if country in saved_dict.keys():
        country = saved_dict[country]
    else:
        print("Please enter correct country name")
    
    input_data = pd.DataFrame(input_data.loc[input_data['country'] == country])
    input_data = input_data.loc[(input_data['year'] > year1 - 1) & (input_data['year'] < year2 + 1)]
    
#     maak van de jaren en maanden kommajaren
    YearMonth = []
    month = []
    for row in input_data['month']:
        month.append(str(row))
        
    count = 0
    for year in input_data['year']:
        year = int(year) + (int(month[count])  - 1)/ 12
        YearMonth.append(year)
        count += 1
    input_data['YearMonth'] = YearMonth
    input_data = input_data[['pr', 'tas', 'country', 'YearMonth']]
    return input_data

select_plot_temperature_and_precipitation(df_weather, 'Lesotho', 1985, 2030)

In [ ]:
df_foodprices

# Plot foodprices per market

In [ ]:
def select_plot_foodprices_per_market(input_data, country, product, year1, year2):
    input_data = input_data.loc[input_data['adm0_name'] == country]
    input_data = input_data.loc[input_data['cm_name'] == product]
    input_data = input_data.loc[(input_data['Year'] >= year1) & (input_data['Year'] < year2 + 1)]
    return input_data

select_plot_foodprices_per_market(df_foodprices, 'Zimbabwe', 'Cowpeas', 1990, 2020)

In [ ]:
# 

def select_plot_foodprices_average(input_data, country, product, year1, year2):
    input_data = select_plot_foodprices_per_market(input_data, country, product, year1, year2)
    
    country_list = []
    product_list = []
    average_price_list = []
    year_list = []
    
    for i in range(year1, year2 + 1):
        for j in range(12):
#             Selecteer een specifieke maand in een jaar
            output_data = input_data.loc[input_data['mp_year'] == i]
            output_data = output_data.loc[output_data['mp_month'] == j + 1]
#         maak een nieuwe rij wanneer er data beschikbaar is
            if len(output_data['mp_price']) > 0:
#                 year
                year_list.append(i + (j / 12))

#                 country
                country_list.append(country)

#                 product
                product_list.append(product)
#                 average data            
                average_price_list.append(sum(output_data['mp_price']) / len(output_data['mp_price']))

    
    output_data = pd.DataFrame()    
    output_data['country'] = country_list
    output_data['year'] = year_list
    output_data['product'] = product_list
    output_data['average_price'] = average_price_list
    return output_data

select_plot_foodprices_average(df_foodprices, 'Zimbabwe', 'Cowpeas', 1998, 2018)    

# Plot functies

In [ ]:
from bokeh.plotting import figure
from bokeh.io import output_file, show, save

import pandas

# Make sure x and y are of the same length.
x1 = select_plot_foodprices_average(df_foodprices, 'Lesotho', 'Beans (sugar-red)', 2004, 2014)['year'] 
y1 = normalize(select_plot_foodprices_average(df_foodprices, 'Lesotho', 'Beans (sugar-red)', 2004, 2014), 'average_price')['Normalized_data']  

x2 = select_plot_temperature_and_precipitation(df_weather, 'Lesotho', 2004, 2014)['YearMonth']    
y2 = normalize(select_plot_temperature_and_precipitation(df_weather, 'Lesotho', 2004, 2014), 'tas')['Normalized_data']    

output_file("Line.html")

f = figure(plot_width=1500, plot_height=600)

# Plot the line
f.line(x1, y1, color='red')
f.line(x2, y2, color='blue')

show(f)

In [ ]:
x1 = normalize(select_plot_migration_movements1(df_migration, 'Zimbabwe', 2004, 2018), 'Value')['Normalized_data']    
y1 = normalize(select_plot_foodprices_average(df_foodprices, 'Zimbabwe', 'Cowpeas', 2004, 2018), 'average_price')['Normalized_data']  

output_file("Line.html")

# Create a figure (that's 600x600)
f = figure(plot_width=600, plot_height=600)
f.xaxis.axis_label = "Temperature and precipitation"
f.yaxis.axis_label = "Average foodprices"

# Plot the line
f.circle(x1, y1, color='red')

show(f)

In [ ]:
def normalize(input_data, column):
    values = []
    
    input_data[column]
    for value in input_data[column]:
         values.append(value)
            
    Min = min(values)
    Max = max(values)
        
    output_data = []
    for value in input_data[column]:
        output_data.append((value - Min)/(Max - Min))
    
    input_data['Normalized_data'] = output_data
    
    return input_data

In [ ]:
def possible_range(data1, data2):
    # bepaal de range van dataset 1 (migration movements)
    min1 = float(data1.head(1))
    max1 = float(data1.tail(1))
    # bepaal de range van dataset 2 (foodprices average)
    min2 = float(data2.head(1))
    max2 = float(data2.tail(1))

    # bepaal de mogelijke combinatie range
    if min1 >= min2: year_min = int(np.ceil(min1))
    else: year_min = int(np.ceil(min2))
    if max1 <= max2: year_max = int(np.ceil(max1))
    else: year_max = int(np.ceil(max1))
        
    return(year_min, year_max)

# werking
# data1 = select_plot_migration_movements1(df_migration, 'Zimbabwe', 1990, 2018)['Timestamp']
# data2 = select_plot_foodprices_average(df_foodprices, 'Zimbabwe', 'Cowpeas', 1990, 2018)['year']
# possible_range(data1, data2)

In [ ]:
# checks if a product is in the foodprices dataset, if not, it returns a list of all available products in a given country
def check_product_in_foodprices(set_country, set_product, foodprices):
    
    set_product = '-' + set_product
    product_list = []
    for product in foodprices.loc[foodprices['adm0_name'] == set_country]['cm_name']:
        if '-' + product not in product_list:
            product_list.append('-' + product)

    d = {'Product_List' : product_list}

    df = pd.DataFrame(data=d)    
    
    if not set_product in product_list:
        print('Error, product not recognized. Choose a product from the list below')
        return df
    else:
        return 'Succesfully imported values!'

In [ ]:
import holoviews as hv
from bokeh.io import show, output_file
from bokeh.models import FactorRange
from bokeh.plotting import figure
import bokeh.layouts
from bokeh.layouts import column
hv.extension('bokeh', 'matplotlib')

def normalise(input_data):

    values = []
    
    for value in input_data:
         values.append(value)
            
    Min = min(values)
    Max = max(values)
        
    output_data = []
    
    for value in input_data:
        output_data.append((value - Min)/(Max - Min))
    
    input_data['Normalized_data'] = output_data

    return input_data

def select_plot_temperature_and_precipitation(input_data, country, year1, year2):
    
    x_range = range(year1,year2)
    
    saved_dict = {'mauritania': 'MRT', 'lesotho': 'LSO', 'somalia': 'SOM', 'nigeria': 'NGA', 'tanzania': 'TZA', 'zambia': 'ZMB', 'burundi': 'BDI', 'afghanistan': 'AFG', 'mali': 'MLI', 'niger': 'NER', 'malawi': 'MWI', 'congo': 'ZAR', 'cabo verde': 'CPV', 'Sudan': 'SDN', 'pakistan': 'PAK', 'burkina faso': 'BFA', 'rwanda': 'RWA', 'kenia': 'KEN', 'senegal': 'SEN', 'cameroon': 'CMR', 'sierra leone': 'SLE', 'iraq': 'IRQ', 'uganda': 'UGA', 'mozambique': 'MOZ', 'zimbabwe': 'ZWE', 'central african republic': 'CAF', 'ethiopia': 'ETH', 'guinea': 'GIN', 'liberia': 'LBR', 'djibouti': 'DJI', 'iran': 'IRN', 'madagascar': 'MDG', 'lebanon': 'LBN'}

    if country in saved_dict.keys():
        country = saved_dict[country]
    else:
        print("Please enter correct country name")
    
    input_data = pd.DataFrame(input_data.loc[input_data['country'] == country])
    input_data = input_data.loc[(input_data['year'] > year1 - 1) & (input_data['year'] < year2 + 1)]
    
#   maak van de jaren en maanden kommajaren
    YearMonth = []
    month = []
    for row in input_data['month']:
        month.append(str(row))
        
    count = 0
    
    for year in input_data['year']:
        
        year = int(year) + (int(month[count])  - 1)/ 12
        YearMonth.append(year)
        count += 1
        
    input_data['YearMonth'] = YearMonth
    input_data = input_data[['pr', 'tas', 'country', 'YearMonth']]
    
    
    return input_data


def bar_graph(input_data, country, year1, year2, target_data1, target_data2):
    
    frame = select_plot_temperature_and_precipitation(input_data, country, year1, year2)

    df1 = normalise(frame[target_data1])
    df2 = normalise(frame[target_data2])
    
    #print(df1)
    
    for cell in df1:
        if type(cell) == list:
            
            data_inject1 = cell
            break
            
    for cell in df2:

        if type(cell) == list:
            
            data_inject2 = cell
            break

    % output size = 250
    % opts Curve (color='red')
    
    bars = hv.Bars(data_inject1, "{}".format(country), 'precipitation and temperature', label = 'precipitation')
    line = hv.Curve(data_inject2, "{}".format(country), 'precipitation and temperature',label = 'temperature')
    
    # function to call food data 
    
    plot = bars * line  
    
    return plot